In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# import tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

# Set device
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
test = pd.read_csv('data/test_set_no_ratings.csv')
# 1. Load Data
train = pd.read_csv('data/train_ratings.csv')
movies = pd.read_csv('data/movies.csv')

links = pd.read_csv('data/links.csv',converters={'imdbId': str, 'tmdbId': str})



In [2]:
movie_db_api_key = "fc13b9e63596b93da9fb0634fcbde491"

In [3]:
# 2. Webscrape
# For each movie in links, get the movie's plot summary from IMDB
# Use BeautifulSoup to parse the html
# Use requests to get the html
# Use re to find the plot summary
# Then use sentence transformer to get the embeddings
import importlib
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import multiprocessing_helper
importlib.reload(multiprocessing_helper)
transformer = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')


# Get the plot summaries for all the movies
# plot_summaries = []
# for link in tqdm(links['tmdbId']):
#     plot_summaries.append(multiprocessing_helper.get_plot_summary(link))

# Multiprocessing
from multiprocessing import Pool
cpu_count = 8
pool = Pool(cpu_count)
# Divide the links into chunks
chunks = np.array_split(links['tmdbId'], cpu_count)
# Get the plot summaries, and keep the order
# helper.get_plot_summary_chunk gets a chunk of links and returns a list of plot summaries
# Each thread will run one helper.get_plot_summary_chunk
plot_summaries = []
plot_summaries.extend(pool.map(multiprocessing_helper.get_plot_summary_chunk, chunks))

/Users/aybarsyazici/Documents/distributed info/-CS-423-Project-2/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Getting plot summaries for 1218 movies
Getting plot summaries for 1218 movies
Getting plot summaries for 1217 movies
Getting plot summaries for 1218 movies
Getting plot summaries for 1218 movies
Getting plot summaries for 1218 movies
Getting plot summaries for 1218 movies
Getting plot summaries for 1217 movies
10% done
10% done
10% done
10% done
20% done
20% done
20% done
20% done
30% done
30% done
30% done
40% done
50% done
40% done
40% done
50% done
60% done
70% done


KeyboardInterrupt: 

In [ ]:
# 3. Pre-processing: Convert genres into a binary encoded vector
movies['genres'] = movies['genres'].str.split('|')
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movies['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_, index=movies.movieId)

# Merge genres data with main data
train = train.merge(genres_df, left_on='movieId', right_index=True)
test = test.merge(genres_df, left_on='movieId', right_index=True)


# 3. Encode users and movies as integer indices
user_enc = LabelEncoder()
train['user_label'] = user_enc.fit_transform(train['userId'])
test['user_label'] = user_enc.transform(test['userId'])